<a href="https://colab.research.google.com/github/jungmin0710/K-digital-PythonDeepLearning/blob/main/Surprise_Package_CPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Python 추천시스템 패키지

In [1]:
import warnings
warnings.filterwarnings('ignore')

## 1.Surprise Package
* 다양한 추천 알고리즘을 쉽게 적용 
* 사이킷런과 유사한 API 구조

### 1) Install Package

In [2]:
!pip install surprise

     |████████████████████████████████| 11.8MB 238kB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1617577 sha256=efc1638def1fbb4c557be6326750fc74accd950a9ae95bb5222f45a8eb00bc08
  Stored in directory: /root/.cache/pip/wheels/78/9c/3d/41b419c9d2aff5b6e2b4c0fc8d25c538202834058f9ed110d0
Successfully built scikit-surprise


### 2) Import Package

In [3]:
import surprise

print(surprise.__version__)

1.1.1


## 2.Surprise Dataset
* user_id(사용자), item_id(아이템), rating(평점)으로 된 데이터 세트만 적용 가능 
* 3개의 컬럼만 로딩하고 나머지는 제외

### 1) MovieLens Dataset
* 디렉토리에 저장 후 로딩
  * 'ml-100k' : 10만개 평점 데이터 'ml-1m' : 
  * 100만개 평점 데이터

In [4]:
from surprise import Dataset 

data = Dataset.load_builtin('ml-100k')

Dataset ml-100k could not be found. Do you want to download it? [Y/n] y
Trying to download dataset from http://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


In [5]:
!ls -l /root/.surprise_data/

total 4
drwxr-xr-x 3 root root 4096 Apr  1 06:48 ml-100k


### 2) train_test_split( )

In [8]:
from surprise.model_selection import train_test_split 

trainset, testset = train_test_split(data,
                                     test_size = 0.3, 
                                     random_state = 2045)



## 3.SVD 기반 잠재 요인 협업 필터링

### 1) fit() : 추천 알고리즘 학습
* SVD(Singular Vector Decomposition)

In [9]:
from surprise import SVD 

algo = SVD()
algo.fit(trainset)

### 2) test( )
* 사용자-아이템 평점 데이터 세트 전체에 대한 추천을 예측 
  * uid : 사용자 아이디 
  * iid : 영화(아이템) 아이디 
  * r_ui : 실제 평점
  * est : Suprise 추천 예측 평점 
  * details : 처리 결과 로그(True-예측값 생성할 수 없는 데이터)

In [10]:
predictions = algo.test(testset)

print('prediction type :',type(predictions), ' size:',len(predictions)) 
print('\n', 'prediction 결과의 최초 5개 추출', '\n') 
predictions[:5]

prediction type : <class 'list'>  size: 30000

 prediction 결과의 최초 5개 추출 



[Prediction(uid='13', iid='531', r_ui=3.0, est=3.2694256681552556, details={'was_impossible': False}),
 Prediction(uid='567', iid='246', r_ui=4.0, est=3.944752879358439, details={'was_impossible': False}),
 Prediction(uid='243', iid='1148', r_ui=3.0, est=3.289034348865252, details={'was_impossible': False}),
 Prediction(uid='346', iid='241', r_ui=4.0, est=3.5344264348599697, details={'was_impossible': False}),
 Prediction(uid='868', iid='1285', r_ui=2.0, est=2.3507029137921744, details={'was_impossible': False})]

* 'uid', 'iid', 'est' 값 추출

In [12]:
[(pred.uid, pred.iid, pred.est) for pred in predictions[:3]]

[('13', '531', 3.2694256681552556),
 ('567', '246', 3.944752879358439),
 ('243', '1148', 3.289034348865252)]

### 3) predict( )
* 개별 사용자의 아이템에 대한 추천 평점 예측 
  * 'uid', 'iid'는 문자열로 입력 
  * 'r_ui' : 기존 평점 정보는 선택 사항
* test( )는 모든 사용자와 아이템에 대해서 predict( )를 반복적으로 수행한 결과

In [13]:
uid = str(196) 
iid = str(302)

pred = algo.predict(uid, iid) 
print(pred)

user: 196        item: 302        r_ui = None   est = 3.96   {'was_impossible': False}


### 4) rmse( )
* 예측 평점과 실제 평점과의 오차 평가

In [14]:
from surprise import accuracy 

accuracy.rmse(predictions)

RMSE: 0.9382


0.9382321956179803

## 4.Data Preprocessing

### 1) user_id(사용자), item_id(아이템), rating(평점)
* 컬럼 Header 제거 필요

In [15]:
import pandas as pd

rurl = 'https://raw.githubusercontent.com/rusita-ai/pyData/master/ratings.csv' 
ratings = pd.read_csv(rurl) 

ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


### 2) index 와 header제거된 파일 생성

In [16]:
ratings.to_csv('ratings_noh.csv', 
               index = False, 
               header = False)

### 3) Suprise - Reader( )
* line_format : 컬럼의 순서 나열 
* sep : 컬럼 구분자 
* rating_scale : 평점 단위를 0.5(최소) ~ 5(최대)로 설정

In [17]:
from surprise import Reader

reader = Reader(line_format = 'user item rating timestamp', 
                sep = ',',
                rating_scale = (0.5, 5))

data = Dataset.load_from_file('ratings_noh.csv', 
                              reader = reader)

### 4) SDV 테스트
* n_factors : 잠재 요인(K) 크기 Hyperparameter

In [18]:
trainset, testset = train_test_split(data,
                                     test_size = .3, 
                                     random_state = 2045)

algo = SVD(n_factors = 50, 
           random_state = 2045)
algo.fit(trainset)

predictions = algo.test(testset) 

accuracy.rmse(predictions)

RMSE: 0.8711


0.871106664601276

## 5.pandas DataFrame
* 판다스 DataFrame에서 데이터 로딩 
  * Dataset.load_from_df( )

In [20]:
from surprise import Reader, Dataset

ratings = pd.read_csv(rurl) 
reader = Reader(rating_scale = (0.5, 5.0))

# ratings DataFrame 에서 컬럼은 사용자 아이디, 아이템 아이디, 평점 순서 준수 
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

trainset, testset = train_test_split(data,
                                     test_size = .3, 
                                     random_state = 2045)

algo = SVD(n_factors = 50, 
           random_state = 2045)
algo.fit(trainset)

predictions = algo.test( testset ) 

accuracy.rmse(predictions)

RMSE: 0.8711


0.871106664601276

## 6.Cross Validation
* cross_validate( )

In [21]:
from surprise.model_selection import cross_validate 

ratings = pd.read_csv(rurl) 
reader = Reader(rating_scale = (0.5, 5.0)) 
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], 
                            reader)

algo = SVD(random_state = 2045)

cross_validate(algo, data,
               measures = ['rmse', 'mae'], 
               cv = 5, verbose = True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8733  0.8659  0.8739  0.8818  0.8781  0.8746  0.0053  
MAE (testset)     0.6690  0.6676  0.6699  0.6798  0.6724  0.6717  0.0043  
Fit time          4.68    4.67    4.65    4.65    4.61    4.65    0.02    
Test time         0.19    0.26    0.29    0.16    0.27    0.23    0.05    


{'fit_time': (4.67571234703064,
  4.665410041809082,
  4.653373718261719,
  4.649456739425659,
  4.606334447860718),
 'test_mae': array([0.6690326 , 0.6675828 , 0.66993967, 0.67977827, 0.67238071]),
 'test_rmse': array([0.87328872, 0.8659012 , 0.87393893, 0.8817761 , 0.87810971]),
 'test_time': (0.1930227279663086,
  0.2616274356842041,
  0.28951382637023926,
  0.1601715087890625,
  0.2689847946166992)}

## 7.Hyperparameter Tuning
* GridSearchCV( )

In [22]:
%%time 

from surprise.model_selection import GridSearchCV

# 하이퍼파라미터 딕셔너리 형태 지정 
# 'n_epochs' : SGD 수행 시 반복 횟수 
# 'n_factors' : 잠재 요인(K) 크기 
param_grid = {'n_epochs': [20, 40, 60], 
              'n_factors': [50, 100, 200]}

gs = GridSearchCV(SVD,
                  param_grid, 
                  measures = ['rmse', 'mae'], 
                  cv = 3)
gs.fit(data)

CPU times: user 4min, sys: 585 ms, total: 4min 1s
Wall time: 4min 1s


* 결과 확인

In [24]:
# 최저 RMSE 점수 
print(gs.best_score['rmse'])

# 최적 하이퍼파라미터 조합 
print(gs.best_params['rmse'])

0.8780853146466359
{'n_epochs': 20, 'n_factors': 50}


## 8. 개인화 영화 추천

### 1) Train Dataset
* ''ratings_noh.csv'

In [25]:
from surprise import Reader 
from surprise.dataset import DatasetAutoFolds

reader = Reader(line_format = 'user item rating timestamp', 
                sep = ',',
                rating_scale = (0.5, 5))

# 'ratings_noh.csv' 파일로 DatasetAutoFolds 클래스 생성 
data_folds = DatasetAutoFolds(ratings_file = 'ratings_noh.csv', reader = reader)

# 전체 데이터를 학습데이터로 생성 
trainset = data_folds.build_full_trainset()

### 2) 영화 정보 확인 
* 사용자가 아직 평점을 매기지 않은 영화
  * 'userId' == 9 
  * 'movieId' == 42

In [26]:
murl = 'https://raw.githubusercontent.com/rusita-ai/pyData/master/movies.csv' 
movies = pd.read_csv(murl)

# userId = 9 의 movieId 데이터 추출 
# movieId = 42 데이터 확인 
movieIds = ratings[ratings['userId'] == 9]['movieId']

if movieIds[movieIds == 42].count() == 0: 
  print('사용자 아이디 9는 영화 아이디 42의 평점 없음', '\n')

print(movies[movies['movieId'] == 42])

사용자 아이디 9는 영화 아이디 42의 평점 없음 

    movieId                   title              genres
38       42  Dead Presidents (1995)  Action|Crime|Drama


### 3) SVD - fit()
* 추천 알고리즘 학습

In [27]:
algo = SVD(n_epochs = 20, 
           n_factors = 50, 
           random_state = 2045)
algo.fit(trainset)

### 4) SVD - predict( )
* 개별 사용자의 아이템에 대한 추천 평점 예측(est = 2.96) 
  * 'uid', 'iid'는 문자열로 입력 
  * 'r_ui' : 기존 평점 정보는 선택 사항

In [28]:
uid = str(9) 
iid = str(42)

pred = algo.predict(uid, iid, verbose = True)

user: 9          item: 42         r_ui = None   est = 2.96   {'was_impossible': False}


### 5) get_unseen_surprise( )
* 사용자가 평점을 주지 않은 영화 목록을 반환 
  * 사용자가 이미 평점을 준 영화 목록을 제거

In [30]:
def get_unseen_surprise(ratings, movies, userId): 
  # 'userId' 사용자가 평점을 매긴 모든 영화 리스트 생성 
  seen_movies = ratings[ratings['userId'] == userId]['movieId'].tolist()

  # 모든 영화 movieId 리스트 생성 
  total_movies = movies['movieId'].tolist()

  # 모든 영화 movieId 중 이미 평점을 매긴 영화의 movieId를 제외하고 리스트 생성 
  unseen_movies = [movie for movie in total_movies if movie not in seen_movies]

  print(' 평점 매긴 영화수 :' , len(seen_movies), '\n', '추천대상 영화수 :', len(unseen_movies), '\n', '전체 영화수:',len(total_movies))

  return unseen_movies 

unseen_movies = get_unseen_surprise(ratings, movies, 9)

 평점 매긴 영화수 : 46 
 추천대상 영화수 : 9696 
 전체 영화수: 9742


### 6) recomm_movie_by_surprise( )
* 최종적으로 사용자에게 영화를 추천 
  * top-10

In [31]:
def recomm_movie_by_surprise(algo, userId, unseen_movies, top_n = 10):
  # predict() 를 평점이 없는 영화에 반복 수행한 후 결과를 List 객체로 저장 
  predictions = [algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]
  
  # predictions List 객체는 surprise의 Predictions 객체를 원소로 가지고 있음 
  # [Prediction(uid='9', iid='1', est=3.69), Prediction(uid='9', iid='2', est=2.98)] 
  # 'est' 값으로 정렬하기 위해서 sortkey_est( ) 함수 정의 
  # sortkey_est( ) 함수는 List 객체의 sort( ) 함수의 키 값으로정렬 수행 
  def sortkey_est(pred): 
    return pred.est

  # sortkey_est( ) 반환값의 내림 차순으로 정렬하고 top_N개의 최상위 값 추출 
  predictions.sort(key = sortkey_est, reverse = True) 
  top_predictions = predictions[:top_n]

  # top_N으로 추출된 영화의 정보 추출
  # 영화 아이디, 추천 예상 평점, 영화 제목 
  top_movie_ids = [int(pred.iid) for pred in top_predictions] 
  top_movie_rating = [pred.est for pred in top_predictions] 
  top_movie_titles = movies[movies.movieId.isin(top_movie_ids)]['title'] 
  top_movie_preds = [(id, title, rating) for id, title, rating in zip(top_movie_ids, top_movie_titles, top_movie_rating)]
  
  return top_movie_preds

### 7) 최종 추천 결과

In [32]:
top_movie_preds = recomm_movie_by_surprise(algo, 
                                           9,
                                           unseen_movies, 
                                           top_n = 10)

print('##### top-10 추천 영화 리스트 #####', '\n') 

for top_movie in top_movie_preds: 
  print(top_movie[1], ':', top_movie[2])

##### top-10 추천 영화 리스트 ##### 

Pulp Fiction (1994) : 4.292320110925793
Shawshank Redemption, The (1994) : 4.280575006684376
Schindler's List (1993) : 4.223054673027752
Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964) : 4.191859292988541
Godfather, The (1972) : 4.191330334239697
Rear Window (1954) : 4.175787864055172
Monty Python and the Holy Grail (1975) : 4.167985211593734
Lawrence of Arabia (1962) : 4.139061529698701
Goodfellas (1990) : 4.133207055423318
Fight Club (1999) : 4.131297142412626


본 내용은 2021/04/01(목)에 학습한 내용입니다.